In [4]:
def kahan_sum(a):
    s = 0
    c = 0
    for i in range(len(a)):
        y = a[i] - c
        t = s + y
        c = (t - s) - y
        s = t
    return s

In [5]:
# also https://code.activestate.com/recipes/393090/
def frsum(iterable):
    return float(sum(imap(Fraction.from_float, iterable)))

In [140]:
#https://code.activestate.com/recipes/393090/
def msum(iterable):
    partials = []
    for x in iterable:
        i = 0
        for y in partials:
            if abs(x) < abs(y):
                x, y = y, x
            hi = x + y #Rounded x+y stored in hi
            lo = y - (hi - x)  #roundoff error stored in lo
            if lo:
                partials[i] = lo #keep track of the roundoffs 
                i += 1
            x = hi
        partials[i:] = [x]
    return sum(partials, 0.0)
    

Testing the standard summation method for 3 tests:
1) will fractions give rounding error?
2) what about extremely large numbers?
3) what about extremely small numbers?
4) scientific notation?

In [101]:
sum([0.1]*10) #should be 1

0.9999999999999999

Testing for large numbers, but it depends if they are integers or floats

Integers work fine, even if they are massive

In [119]:
sum([3,10**200,-10**200]*20)  #should be 60

60

Floats don't.'

In [121]:
sum([3.0,10.0**20,-10.0**20]*20) #should be 60

0.0

Scientific notation gives floats, which is why it doesn't work.

In [122]:
sum([3,10e20,-10e20]*20) #should be 60

0.0

My first attempt at improving this was usign a kahan sum, testing with:
1) will fractions give rounding error?
2) will it woork with scientific notation

It fixes improves the accuracy when dealing with decimals

In [125]:
kahan_sum([0.1]*10)   #should be 1.0

1.0

Kahan is still unable to overcome floating point precision error

In [126]:
kahan_sum([3,10e20,-10e20]*20)  #should be 60

0.0

The kahan sum made some improvements. But all it does is keep track of the decimals more closely. Further improved sums:
testing the msum method:
1) will fractions give round error?
2) scientific notation?

In [127]:
msum([0.1]*10) #should be 1.0

1.0

The msum method works for floating point numbers without loss of precision, examples:

In [129]:
msum([3.0,10e100,-10e100]*20)  #should be 60

60.0

In [130]:
msum([3,10.0**100,-10.0**100]*20)   #should be 60

60.0

In [141]:
msum([3, 10.0**100,1e-20,-10.0**100,-1e-20]*20)   #should be 60

60.0

In [142]:
msum([0.1]*10)

1.0

In [155]:
def print_msum(iterable):
    partials = []
    for x in iterable:
        print('Working on {0}'.format(x))
        i = 0
        for y in partials:
            if abs(x) < abs(y):
                x, y = y, x
            hi = x + y #Rounded x+y stored in hi
            print('Hi = {0}'.format(hi))
            lo = y - (hi - x)  #roundoff error stored in lo
            if lo:
                print('Lo = {0}'.format(lo))
                partials[i] = lo #keep track of the roundoffs 
                i += 1
            x = hi
        partials[i:] = [x]
        print('Done with {0}'.format(x))
    print('Done, returning sum')
    print(partials)
    return sum(partials, 0.0)

Print msum will let me see what values are being stored in partials

In [156]:
print_msum([3, 10.0**100,1e-20,-10.0**100,-1e-20])

Working on 3
Done with 3
Working on 1e+100
Hi = 1e+100
Lo = 3.0
Done with 1e+100
Working on 1e-20
Hi = 3.0
Lo = 1e-20
Hi = 1e+100
Lo = 3.0
Done with 1e+100
Working on -1e+100
Hi = -1e+100
Lo = 1e-20
Hi = -1e+100
Lo = 3.0
Hi = 0.0
Done with 0.0
Working on -1e-20
Hi = 0.0
Hi = 3.0
Hi = 3.0
Done with 3.0
Done, returning sum
[3.0]


3.0

Which gave me an idea on how to break it

In [159]:
print_msum([1e-20,1e100]) == 1e100

Working on 1e-20
Done with 1e-20
Working on 1e+100
Hi = 1e+100
Lo = 1e-20
Done with 1e+100
Done, returning sum
[1e-20, 1e+100]


True

I suppose the short-coming here is that the value must be able to be stored in a single float's worth of precision

In [16]:
import math
def pairwise_sum(a):
    n = len(a)
    if n <= 3:
        s = a[0]
        for i in range(1,n):
            s += a[i]
    else:
        m = math.floor(n/2)
        s = pairwise_sum(a[:m]) + pairwise_sum(a[m:])
    return s

In [17]:
pairwise_sum([0.1]*10)

1.0

In [18]:
pairwise_sum([3,10**30,-10**30]*20)

60

In [19]:
pairwise_sum([3,1e20,-1e20]*20)

0.0

Coclusion - Pairwise is good with decimals but cannot handle scientific notation

In [60]:
# Part 2: working with simple distance() method

In [20]:
"""(3) Find an example (an easy one) `(a,b)`,`(0,0)` that makes `distance` go boom."""
def distance(x1,y1,x2,y2):
    return math.sqrt((x2-x1)**2+(y2-y1)**2)

In [118]:
distance(.0001,10000,0,0) #This one will be slightly off, large numbers mess with the precision

10000.0

In [187]:
"""Other distance method can't handle small differences in distances very well"""
from decimal import *
def dist(x1,y1,x2,y2,prec):
    getcontext().prec = prec
    x = Decimal(x2)-Decimal(x1)
    print(x)
    y = Decimal(y2)-Decimal(y1)
    print(y)
    return (x**2+y**2).sqrt()

Using decimal we can get far more precision, setting the context according to our needs

In [192]:
from decimal import *
getcontext().prec = 400
Decimal(1)/Decimal(7)

Decimal('0.1428571428571428571428571428571428571428571428571428571428571428571428571428571428571428571428571428571428571428571428571428571428571428571428571428571428571428571428571428571428571428571428571428571428571428571428571428571428571428571428571428571428571428571428571428571428571428571428571428571428571428571428571428571428571428571428571428571428571428571428571428571428571428571428571428571428571429')

One problem I encountered here is that the error occurs before the values are inputted to the function. In this example, the x distance is wrong because Decimal(0.0001) is accepting 0.0001 as a float, where it's true value in memory is what is shown below.

In [249]:
dist(.0001,10000.0,0,0,60) #should be 10000.0000000000004999999999999999875000000000000 according to wolfram alpha

-0.000100000000000000004792173602385929598312941379845142364501953
-10000


Decimal('10000.0000000000005000000000000000354217360238592953602890044')

The Fix:

In [250]:
def dist_fixed(x1,y1,x2,y2,prec):
    getcontext().prec = prec
    x = x2-x1
    print(x)
    y = y2-y1
    print(y)
    return (getcontext().power(x,2)+getcontext().power(y,2)).sqrt()

In [251]:
x1 = Decimal('.0001')
x2 = Decimal(0)
y1 = Decimal('10000.0')
y2 = Decimal(0)
dist_fixed(x1,y1,x2,y2,30) #should be 10000.0000000000004999999999999999875000000000000 according to wolfram alpha

-0.0001
-10000.0


Decimal('10000.0000000000005000000000000')

It seems broken, but it works if we turn the precision up extremely high. Perhaps I have the context set poorly here.

In [252]:
x1 = Decimal('.0001')
x2 = Decimal(0)
y1 = Decimal('10000.0')
y2 = Decimal(0)
dist_fixed(x1,y1,x2,y2,3000) #should be 10000.0000000000004999999999999999875000000000000 according to wolfram alpha

-0.0001
-10000.0


Decimal('10000.0000000000004999999999999999875000000000000006249999999999999609375000000000027343749999999997949218750000000161132812499999986907958984375001091003417968749907264709472656258008956909179686799216270446777405742406845092767902463674545288584090769290924027120484970510010067102266475557903628825442865525666180474217977885449381574290171744015424337675919711023198035817431549503453181373221774208582422434462770863889093319508916328423631904609417260912119783668626143645008219444258627693038215980315701390047733038576087957531368614148815575846627859722255525352339475691996226113591131547426457607313276037137776006802565550161638371618730068482167342698532645637216613692408420659213639545140897108223795219509885233799469046652545966534895382642933328569679382716013956749629823275738902180219618982840064974644106968719634802020811216167199814871041939686787840649590967710797841035282623779214222961365647914334542664169239604393993804581834027887475788837003298341085452357949

Reading from existing floats doesn't seem to work, unless I did something wrong here

In [253]:
Decimal.from_float(1.1)

Decimal('1.100000000000000088817841970012523233890533447265625')